# Make, Workflow, and GitHub Action

## Introduction

Modern computational astrophysics is no longer just about interacting
with the terminal or writing one-off scripts that generate plots or
results.
Research projects involve complex pipelines:
*i*) generating synthetic data,
*ii*) calibrating instruments,
*iii*) analyzing observations,
*iv*) running simulations, and
*v*) producing figures and papers.
Each step depends on the outputs of earlier steps.
And the whole chain may need to be repeated when code changes, new
data arrive, or collaborators join the project.
Without systematic management, such workflows quickly become fragile,
error-prone, and difficult to reproduce.

Workflow management and automation tools address these challenges.
They allow us to:
* Capture dependencies:
  make sure each step runs only when its inputs are ready.
* Avoid redundant work:
  rebuild only the outputs affected by changes.
* Scale up easily:
  run dozens or thousands of jobs in parallel on HPC or the cloud.
* Enable reproducibility:
  capture all the steps needed to regenerate results.

In practice, we often combine several layers of automation:

* `make`:
  lightweight automation for compiling code, running tests, build
  documentations, and chaining a few steps together.
  `Make` has been around for decades and remains a powerful tool for
  small pipelines.
* Workflow engines (such as `Snakemake`):
  generalizations of `make` that make it easy to run large-scale
  scientific workflows in parallel, track provenance, and use portable
  environments.
* Continuous Integration / Continuous Deployment (CI/CD):
  services such as *GitHub Actions* that automatically run tests,
  build documentation, and execute reproducible mini-workflows every
  time code is shared or updated.
  This ensures that the project stays healthy, reproducible, and
  transparent.

## What You Will Learn

In this lab, we will build a **minimal CCD image calibration package**
and then apply workflow automation to it at multiple levels:
1. Package & Testing:
   Write simple calibration functions in a Python package and test
   them with `pytest`.
2. `make`:
   Automate local development tasks such as testing, linting, and
   running a small pipeline.
3. `Snakemake`:
   Scale up the calibration pipeline to handle many images in parallel.
4. CI/CD with GitHub Actions:
   Automate testing and documentation generation whenever code is
   pushed to GitHub.

By the end of this lab, you will see how automation tools turn
individual scripts into a reproducible scientific workflow that is
easier to run, easier to share, and easier to trust.

## Part 1: Set up a Python Package with Tests

In real research, we rarely write a single script that does
everything.
Instead, we build up small, reusable functions, things like "combine
all bias frames" or "subtract dark current".
Over time, these functions naturally belong in a package: a collection
of modules that can be imported, tested, and reused across multiple
projects.

For this lab, let's create a toy package called `ccdmini`.
Its job is to provide the most basic CCD calibration primitives:
* `median_stack`:
  combine multiple images (e.g., biases, darks, flats) into a single
  master calibration frame by taking the median pixel-by-pixel.
* `make_master_bias`, `make_master_dark`, `make_master_flat`:
  convenience functions that wrap around `median_stack` and perform
  normalizations where needed.
* `apply_calibration`: apply the standard CCD calibration formula:
  \begin{align}
    \text{Calibrated Image} =
    \frac{(\text{Raw Image} - \text{Master Bias} - \text{Master Dark})}{\text{Master Flat}}
  \end{align}

This is the exact same operation astronomers run on real raw CCD frames.
In our case, we will use tiny synthetic arrays to keep things simple
and fast.

### Why packaging?

Even though this is just a mock example, packaging matters because:

* Reusability:
  you can use the same functions in multiple projects or scripts.
* Testability:
  you can isolate and test each function with `pytest`.
* Shareability:
  once it's a package, you could publish it to PyPI, or share it
  within a collaboration with version control.